# Economic Data Science Project 

In [30]:
%pip install fredapi 
%pip install plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

plt.style.use('fivethirtyeight') #Sets the style to fivethirtyeight

from fredapi import Fred

In [4]:
fred_key = 'c9e12ca3af3c2130d3b97e9cf9f2e861'

# 1. Create the Fred Object

In [5]:
fred = Fred(api_key=fred_key) #Creates a fred object that we will use as main object to pull data

# 2. Search for econmic data

In [6]:
sp_search = fred.search('S&P',filter=('frequency', 'Monthly'), limit=100)

In [28]:
sp_search.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
CSUSHPINSA,CSUSHPINSA,2023-02-08,2023-02-08,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2022-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2023-01-31 08:01:02-06:00,90,"For more information regarding the index, plea..."
CSUSHPISA,CSUSHPISA,2023-02-08,2023-02-08,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2022-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Seasonally Adjusted,SA,2023-01-31 08:01:04-06:00,82,"For more information regarding the index, plea..."
SPCS20RSA,SPCS20RSA,2023-02-08,2023-02-08,S&P/Case-Shiller 20-City Composite Home Price ...,2000-01-01,2022-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Seasonally Adjusted,SA,2023-01-31 08:02:02-06:00,70,"For more information regarding the index, plea..."
SPCS20RNSA,SPCS20RNSA,2023-02-08,2023-02-08,S&P/Case-Shiller 20-City Composite Home Price ...,2000-01-01,2022-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2023-01-31 08:02:35-06:00,34,"For more information regarding the index, plea..."
SFXRSA,SFXRSA,2023-02-08,2023-02-08,S&P/Case-Shiller CA-San Francisco Home Price I...,1987-01-01,2022-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Seasonally Adjusted,SA,2023-01-31 08:02:04-06:00,66,"For more information regarding the index, plea..."


In [35]:
unemp_rate = fred.search('Unemployment', order_by= 'popularity', limit=100)

In [36]:
unemp_rate = fred.get_series('UNRATE', observation_start='1990-01-01', observation_end='2020-12-31')

In [135]:
#Convert to dataframe and relabel columns
unemp_rate =pd.DataFrame(unemp_rate)
unemp_rate = unemp_rate.reset_index()
unemp_rate.columns = ['Date', 'Unemployment Rate']

In [142]:
px.line(unemp_rate, x='Date', y='Unemployment Rate', title='Unemployment Rate')

# 3.Pulling Raw Data & Plot

In [8]:
home_price_index = fred.get_series('CSUSHPINSA')

In [9]:
home_price= home_price_index

In [10]:
home_price

1975-01-01        NaN
1975-02-01        NaN
1975-03-01        NaN
1975-04-01        NaN
1975-05-01        NaN
               ...   
2022-07-01    307.164
2022-08-01    303.676
2022-09-01    300.549
2022-10-01    298.958
2022-11-01    297.290
Length: 575, dtype: float64

In [11]:
home_df = pd.DataFrame(home_price, columns=['Home Price Index'])

In [37]:

home_df.index = pd.to_datetime(home_df.index, format='%Y-%m-%d')

In [39]:
home_df.dropna(inplace=True)

In [44]:
#Rename index column as date 
home_df.index.name = 'Date'


In [45]:
px.bar(home_df, x=home_df.index,  y='Home Price Index', title='Home Price Index Over Time')

# 4. Pull and Join Multiple Data Series

In [93]:
fred.search("Inflation", filter=('frequency', 'Monthly'), order_by = 'popularity', limit=50)
inflation = fred.get_series('MICH', observation_start='1990-01-01', observation_end='2020-12-31')

inflation_df = pd.DataFrame(inflation, columns=['Inflation'])
inflation_df.index = pd.to_datetime(inflation_df.index, format='%Y-%m-%d')

In [95]:
inflation_df.reset_index(inplace=True)

In [100]:
inflation_df.rename(columns = {'index':'Date'})

,Date,Inflation
0,1990-01-01,4.1
1,1990-02-01,4.1
2,1990-03-01,3.7
3,1990-04-01,3.6
4,1990-05-01,3.4
...,...,...
367,2020-08-01,3.1
368,2020-09-01,2.6
369,2020-10-01,2.6
370,2020-11-01,2.8


In [103]:
px.line(inflation_df, x='index', y='Inflation', title='Inflation Over Time')

In [105]:
#Create a dataframe with all the data
df = pd.DataFrame({'Unemployment Rate': unemp_rate, 'Home Price Index': home_price, 'Inflation': inflation})


Clean Null Values 

In [107]:
df.dropna(inplace=True)

Make Date its own series and replace with an index

In [115]:
df.reset_index(inplace=True)
df.set_axis(['Date', 'Unemployment Rate', 'Home Price Index', 'Inflation'], axis= 'columns' , copy=False)
df

,Date,Unemployment Rate,Home Price Index,Inflation
0,1990-01-01,5.4,76.527,4.1
1,1990-02-01,5.3,76.587,4.1
2,1990-03-01,5.2,76.790,3.7
3,1990-04-01,5.4,77.038,3.6
4,1990-05-01,5.4,77.296,3.4
...,...,...,...,...
367,2020-08-01,8.4,224.074,3.1
368,2020-09-01,7.9,226.832,2.6
369,2020-10-01,6.9,229.851,2.6
370,2020-11-01,6.7,232.364,2.8


# Plot Data from Combined Table 


In [116]:
px.line(df, x='Date', y=['Unemployment Rate','Inflation'], title='Unemployment Rate and Inflation Over Time')

In [122]:
fig = px.line(df, x='Date', y='Inflation', title='Inflation with Range Slider')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()
